# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
# Passo 1 - Entrar na internet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = webdriver.Chrome(options=chrome_options)

navegador = webdriver.Chrome()

# Passo 2 - Pegar cotação do dólar
#entrar no google
navegador.get('https://www.google.com/')
#pesquisar 'cotação do dolar'
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação dólar')
# para encontrar o xpath tem q abrir a página> 
# Rclick>inspecionar>Lclick(no símbolo lá em cima q tem desenhado um cursor)> passa o mouse em cima de onde vc quer
# > Lclick > Rclick(no codigo q ficou azul no lado direito) > Rclick > copy > copy xpath

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# pegar o número que aparece no resultado do google
cotacao_dolar = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Passo 3 - Pegar cotação do euro
#entrar no google
navegador.get('https://www.google.com/')
#pesquisar 'cotação do dolar'
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação euro')

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# pegar o número que aparece no resultado do google
cotacao_euro = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Passo 4 - Pegar cotação do ouro
#entrar no google
navegador.get('https://www.melhorcambio.com/ouro-hoje')
#pega a cotação no campo do site
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')

navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
import pandas as pd

bd = pd.read_excel('Produtos.xlsx')
bd

- Atualizando os preços e o cálculo do Preço Final

In [ ]:
# atualizar a cotação
bd.loc[(bd.Moeda == 'Dólar'),'Cotação'] = float(cotacao_dolar)
bd.loc[(bd.Moeda == 'Euro'),'Cotação'] = float(cotacao_euro)
bd.loc[(bd.Moeda == 'Ouro'),'Cotação'] = float(cotacao_ouro)

# atualizar Preço base Reais
bd['Preço Base Reais'] = bd.Cotação * bd['Preço Base Original']

# atualizar Preço Final
bd['Preço Final'] = bd.Margem * bd['Preço Base Reais']

#mudar  o número de casas decimais
bd['Cotação'] = bd['Cotação'].map('{:.2f}'.format)
bd['Preço Base Reais'] = bd['Preço Base Reais'].map('{:.2f}'.format)
bd['Preço Final'] = bd['Preço Final'].map('{:.2f}'.format)
display(bd)

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
# Passo 6 - Exportar a base de dados
bd.to_excel('BaseAtualizada.xlsx', index=False)
#index=False para n colocar no novo arquivo uma coluna com os índices q o pandas gera automaticamente